# Where should I open a restaurant at Polish seaside?
# 1. Introduction

## 1.1 Background
Over past years polish seaside towns have significantly developed in terms of holiday housing for rent or summer appartments. This trend on housing market has been followed by development of local venues. It is however not clear now if the market is already saturated and many entrepreneurs will wonder which are the best resorts to invest in, if one wants to open a restaurant. It may also not be clear if a particular type of cuisine is popular and there's still opportunity to open new venues of that type with positive return on investment.

Recent restrictions on restaurants and public places imposed due to virus threat also point to additional challanges in this industry, which should be considered in terms of business model. A company needs to have more resiliant offer, which would allow it to survive in times of crisis like this. A well rounded and diversified offer towards the public is preferred, over a standard fish&chips van. Reading through available research and publications is a must for entrepreneurs to make sure they exhaust information sources to guide their investment decisions.

This report could also be treated as an objective context or argument when entrepreneurs try to get external funding for launch of their business in these extraordinary times. Beyond entrepreneurs, the content of this project will be valuable to capital funds, venture capitalists or business angels, who invest private money into businesses. It will present and assess overall potential of polish seaside resorts based on objective, data-based criteria for Italian restaurants.

## 1.2 Problem

In this report, I will narrow down the problem to answer the following question: where should an entrepreneur open an Italian cuisine restaurant, considering Polish seaside resorts. Among factors I intend to consider will be number of restaurants, their cuisine type, their geospatial distribution in selected towns. 
Working assumption is that a place which is good to open a new restaurant does not have a neighboring restaurant of the same cuisine type in range of 15km. It should however be in a "food-serving" zone of the town so it's easily accessible to people. If there is no other Italian restaurant in certain town, this town should be considered a 'golden spot' location.

# 2. Data acquisition and cleaning

### 2.1. Data sources

#### 2.1.2. Location data
I will use foursquare data (https://foursquare.com/) to get information about seaside resorts in Poland. Foursquare has plenty of data points about places, globally - 65M+ from 190 Countries. 900 Venue categories - including restaurants and cuisine type, including 'Italian restaurant' which will be of interest for this project (https://developer.foursquare.com/docs/build-with-foursquare/categories/). 30+ attributes for each of those venues, where most important here will be venue name, address, ratings, and reviews. Among others, it's possible in this data set to check visits to selected locations, which would be relevant when deciding about location of a new restaurant. Foursquare enables identification of popular and trending location, which I will also include.

#### 2.1.3. Municipalities - administrative split

I will preselect the places to include into my comparative analysis, as I am only interested in major seaside resorts. To ensure the analysis is useful to wide spectrum of audience I will include all municipalities which are directly by the sea and in western Poland. The list will be compiled from here: https://upload.wikimedia.org/wikipedia/commons/f/f9/Zachodniopomorskie_powiaty_2005.svg. The following towns will be included: Świnoujście, Międzyzrdoje, Dziwnów, Rewal, Kołobrzeg, Ustronie Morskie, Mielno, Darłowo, Postomino.



In [1]:
#defining collection of selected seaside resorts
towns = ['Świnoujście', 'Międzyzrdoje', 'Dziwnów', 'Rewal', 'Kołobrzeg', 'Ustronie Morskie', 'Mielno', 'Darłowo', 'Postomino']
towns_en = ['Swinoujscie', 'Miedzyzrdoje', 'Dziwnow', 'Rewal', 'Kolobrzeg', 'Ustronie Morskie', 'Mielno', 'Darlowo', 'Postomino']

### 2.2. Data cleaning
#### 2.2.1. Importing and installing needed python libraries

In [2]:
#Installing and/or importing necessary libraries
import pandas as pd
import numpy as np
print('Basics on board!')

Basics on board!


In [3]:
# library to handle JSON files
import json 

# library to handle requests
import requests 
from pandas.io.json import json_normalize 

In [4]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

print('Ready to draw :)')

Ready to draw :)


In [5]:
# import k-means for clustering
from sklearn.cluster import KMeans

In [ ]:
# import map rendering library
!conda install -c conda-forge folium=0.5.0 --yes
print('Folium installed')

!pip install geopy
print('Geopy installed')

import folium 
from geopy.geocoders import Nominatim
print('Nominatim is with us')

print('Ready to show Maps!')

Solving environment: - 

In [ ]:
# Forsquare API - my credentials
CLIENT_ID = 'APHKRAH5WDSPA3LBL5MKKF13JEKSTBYT501C02U3Z31TIF5I' # your Foursquare ID
CLIENT_SECRET = 'GQY5SUH5A2MTYJRBAB4JJPFCPWSC5N1P0TPM5DWAQB5FSHKA' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

#### 2.2.2 Extracting longitude and latitude of western seaside resorts

In [48]:
towns_en = ['Swinoujscie', 'Miedzyzrdoje', 'Dziwnow', 'Rewal', 'Kolobrzeg', 'Ustronie Morskie', 'Mielno', 'Darlowo', 'Postomino']
locdata = pd.DataFrame(columns=['Url', 'Longitude', 'Latitude'])
locdata

,Url,Longitude,Latitude


In [278]:
locdata = pd.DataFrame(columns=['Url', 'Latitude', 'Longitude','Address'])
address = ['Świnoujście, Poland','Międzyzdroje, Poland','Dziwnow, Poland', 'Rewal, Poland', 'Kołobrzeg, Poland','Ustronie Morskie, Poland', 'Mielno, Poland','Darlowo, Poland', 'Postomino, Poland' ]
i = 0
while i < 9:
    #identifying longitude and latitude of each of the towns
    geolocator = Nominatim(user_agent="ny_explorer")
    location = geolocator.geocode(address[i])
    latitude = location.latitude
    longitude = location.longitude
    # Forsquare - create URL
    LIMIT = 100 # limit of number of venues returned by Foursquare API
    radius = 2000 # define radius
  
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        latitude, 
        longitude, 
        radius, 
        LIMIT)
       
    locdata = locdata.append({'Url': url, 'Latitude': latitude, 'Longitude': longitude,'Address': address[i]}, ignore_index=True)
    i += 1 
    
locdata

,Url,Latitude,Longitude,Address
0,https://api.foursquare.com/v2/venues/explore?&...,53.909748,14.251070,"Świnoujście, Poland"
1,https://api.foursquare.com/v2/venues/explore?&...,53.928275,14.448903,"Międzyzdroje, Poland"
2,https://api.foursquare.com/v2/venues/explore?&...,54.025095,14.752647,"Dziwnow, Poland"
3,https://api.foursquare.com/v2/venues/explore?&...,54.082067,15.014193,"Rewal, Poland"
4,https://api.foursquare.com/v2/venues/explore?&...,54.175961,15.576421,"Kołobrzeg, Poland"
5,https://api.foursquare.com/v2/venues/explore?&...,54.213598,15.756076,"Ustronie Morskie, Poland"
6,https://api.foursquare.com/v2/venues/explore?&...,54.259006,16.054000,"Mielno, Poland"
7,https://api.foursquare.com/v2/venues/explore?&...,54.422490,16.409330,"Darlowo, Poland"
8,https://api.foursquare.com/v2/venues/explore?&...,54.492360,16.713720,"Postomino, Poland"


In [132]:
locdata['Url'][0]

'https://api.foursquare.com/v2/venues/explore?&client_id=APHKRAH5WDSPA3LBL5MKKF13JEKSTBYT501C02U3Z31TIF5I&client_secret=GQY5SUH5A2MTYJRBAB4JJPFCPWSC5N1P0TPM5DWAQB5FSHKA&v=20180605&ll=53.9097477,14.2510703&radius=2000&limit=100'

In [133]:
json_data = pd.DataFrame(columns=['TownJson'])
i=0
while i<9:
    results = requests.get(locdata['Url'][i]).json()
    json_data = json_data.append({'TownJson': results}, ignore_index=True)
    i += 1
json_data

,TownJson
0,"{'meta': {'code': 200, 'requestId': '5f077c24b..."
1,"{'meta': {'code': 200, 'requestId': '5f077ec53..."
2,"{'meta': {'code': 200, 'requestId': '5f077da9a..."
3,"{'meta': {'code': 200, 'requestId': '5f077d5d3..."
4,"{'meta': {'code': 200, 'requestId': '5f077d5c5..."
5,"{'meta': {'code': 200, 'requestId': '5f077e14a..."
6,"{'meta': {'code': 200, 'requestId': '5f077d8b6..."
7,"{'meta': {'code': 200, 'requestId': '5f077d8c7..."
8,"{'meta': {'code': 200, 'requestId': '5f077d98a..."


In [220]:
print(json_data['TownJson'][8])
#Conclusion - There are no venues in Postomino

{'meta': {'code': 200, 'requestId': '5f077d98a45aef3f3e51d135'}, 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."}, 'headerLocation': 'Current map view', 'headerFullLocation': 'Current map view', 'headerLocationGranularity': 'unknown', 'totalResults': 0, 'suggestedBounds': {'ne': {'lat': 54.510360018000014, 'lng': 16.744653288479228}, 'sw': {'lat': 54.47435998199998, 'lng': 16.68278671152077}}, 'groups': [{'type': 'Recommended Places', 'name': 'recommended', 'items': []}]}}


In [52]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

### Świnoujście

In [198]:
#Świnoujście
i=0
# listing the venues as pandas dataframe
venues = json_data['TownJson'][i]['response']['groups'][0]['items']
SWI_nearby_venues = json_normalize(venues) # flatten JSON
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng','venue.location.city']
SWI_nearby_venues = SWI_nearby_venues.loc[:, filtered_columns]
# filter the category for each row
SWI_nearby_venues['venue.categories'] = SWI_nearby_venues.apply(get_category_type, axis=1)
# clean columns
SWI_nearby_venues.columns = [col.split(".")[-1] for col in SWI_nearby_venues.columns]
#Substituting NaN 
SWI_nearby_venues['city'] = SWI_nearby_venues['city'].fillna('Świnoujście')
SWI_nearby_venues

,name,categories,lat,lng,city
0,Plac Wolności,Plaza,53.909011,14.248993,Świnoujście
1,Promenada,Plaza,53.918722,14.259256,Świnoujście
2,Hampton by Hilton,Hotel,53.913719,14.236786,Świnoujście
3,Plaża Miejska,Beach,53.921737,14.253880,Świnoujście
4,Magiczna Spiżarnia,Polish Restaurant,53.907843,14.255107,Świnoujście
5,Amsterdam Café,Restaurant,53.919272,14.256897,Świnoujście
6,Radisson Blu Resort Świnoujście,Resort,53.919284,14.260361,Świnoujście
7,Park Zdrojowy,Park,53.912908,14.258346,Świnoujście
8,Neptun,Eastern European Restaurant,53.909136,14.249110,Świnoujście
9,Mila,Restaurant,53.919168,14.258201,Świnoujście


### Międzyzdroje

In [202]:
#Międzyzdroje
i=1
# listing the venues as pandas dataframe
venues = json_data['TownJson'][i]['response']['groups'][0]['items']
MIE_nearby_venues = json_normalize(venues) # flatten JSON
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng','venue.location.city']
MIE_nearby_venues = MIE_nearby_venues.loc[:, filtered_columns]
# filter the category for each row
MIE_nearby_venues['venue.categories'] = MIE_nearby_venues.apply(get_category_type, axis=1)
# clean columns
MIE_nearby_venues.columns = [col.split(".")[-1] for col in MIE_nearby_venues.columns]
#Substituting NaN 
MIE_nearby_venues['city'] = MIE_nearby_venues['city'].fillna('Międzyzdroje')
MIE_nearby_venues

,name,categories,lat,lng,city
0,Tawerna Róża Wiatrów,Restaurant,53.928670,14.450467,Międzyzdroje
1,Plaża w Międzyzdrojach,Beach,53.930155,14.446247,Międzyzdroje
2,Molo,Pier,53.931688,14.446175,Międzyzdroje
3,Park Chopina,Park,53.931023,14.449581,Międzyzdroje
4,Restauracja Carmen Międzyzdroje,Pizza Place,53.929117,14.450588,Międzyzdroje
5,Promenada Gwiazd Międzyzdroje,Pedestrian Plaza,53.932556,14.449904,Międzyzdroje
6,Woliński Park Narodowy (National Park),National Park,53.933363,14.459106,Międzyzdroje
7,Inna bajka,Dessert Shop,53.935204,14.455413,Międzyzdroje
8,Nemo Bistro&Pub,Seafood Restaurant,53.931478,14.447263,Międzyzdroje
9,Port,Fish & Chips Shop,53.936860,14.457846,Międzyzdroje


### Dziwnow

In [203]:
#Dziwnow
i=2
# listing the venues as pandas dataframe
venues = json_data['TownJson'][i]['response']['groups'][0]['items']
DZI_nearby_venues = json_normalize(venues) # flatten JSON
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng','venue.location.city']
DZI_nearby_venues = DZI_nearby_venues.loc[:, filtered_columns]
# filter the category for each row
DZI_nearby_venues['venue.categories'] = DZI_nearby_venues.apply(get_category_type, axis=1)
# clean columns
DZI_nearby_venues.columns = [col.split(".")[-1] for col in DZI_nearby_venues.columns]
#Substituting NaN 
DZI_nearby_venues['city'] = nearby_venues['city'].fillna('Dziwnow')
DZI_nearby_venues

,name,categories,lat,lng,city
0,Plaża Dziwnów,Beach,54.026482,14.751197,Kołobrzeg
1,Złota Rybka,Seafood Restaurant,54.025232,14.753585,Kołobrzeg
2,Port rybacki Dziwnów,Harbor / Marina,54.024209,14.753913,Kołobrzeg
3,Nadmorski Park Miniatur i Kolejek,Park,54.018232,14.744697,Kołobrzeg
4,Hotel Stary Dziwnów,Resort,54.028020,14.767321,Kołobrzeg


### Rewal

In [206]:
i=3
# listing the venues as pandas dataframe
venues = json_data['TownJson'][i]['response']['groups'][0]['items']
REW_nearby_venues = json_normalize(venues) # flatten JSON
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng','venue.location.city']
REW_nearby_venues = REW_nearby_venues.loc[:, filtered_columns]
# filter the category for each row
REW_nearby_venues['venue.categories'] = REW_nearby_venues.apply(get_category_type, axis=1)
# clean columns
REW_nearby_venues.columns = [col.split(".")[-1] for col in REW_nearby_venues.columns]
#Substituting NaN 
REW_nearby_venues['city'] = REW_nearby_venues['city'].fillna('Rewal')
REW_nearby_venues

,name,categories,lat,lng,city
0,Plaża Rewal,Beach,54.081997,15.012069,Rewal
1,Ruiny Kościoła w Trzęsaczu,Other Great Outdoors,54.077350,14.992405,Rewal
2,Cafe Patio,Café,54.081245,15.013800,Rewal
3,Caffe Sorrento,Café,54.082001,15.013276,Rewal
4,"Smażalnia ryb ""U Wita""",Seafood Restaurant,54.081452,15.011146,Rewal
5,Tawerna Rybacka,Seafood Restaurant,54.082283,15.010828,Rewal
6,Rybio Grande,Fish & Chips Shop,54.081685,15.009358,Rewal
7,FruitBar,Juice Bar,54.080449,15.007370,Rewal
8,Polo Market Rewal,Grocery Store,54.078670,15.009227,Rewal
9,Shell,Gas Station,54.077119,15.032761,Sliwin


### Kołobrzeg

In [209]:
i=4
# listing the venues as pandas dataframe
venues = json_data['TownJson'][i]['response']['groups'][0]['items']
KOL_nearby_venues = json_normalize(venues) # flatten JSON
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng','venue.location.city']
KOL_nearby_venues = KOL_nearby_venues.loc[:, filtered_columns]
# filter the category for each row
KOL_nearby_venues['venue.categories'] = KOL_nearby_venues.apply(get_category_type, axis=1)
# clean columns
KOL_nearby_venues.columns = [col.split(".")[-1] for col in KOL_nearby_venues.columns]
#Substituting NaN 
KOL_nearby_venues['city'] = KOL_nearby_venues['city'].fillna('Kołobrzeg')
KOL_nearby_venues

,name,categories,lat,lng,city
0,Domek Kata,Eastern European Restaurant,54.176595,15.576488,Kołobrzeg
1,Kołobrzeg - Plaża wschodnia,Beach,54.186733,15.583945,Kołobrzeg
2,Hotel AQUARIUS SPA,Hotel,54.183829,15.591086,Kołobrzeg
3,Pirania 2,Seafood Restaurant,54.184779,15.565897,Kołobrzeg
4,W Filiżance Cafe,Café,54.175504,15.574885,Kołobrzeg
5,Kawiarnia Coffeedesk,Coffee Shop,54.166245,15.557303,Kołobrzeg
6,Latarnia Morska Kołobrzeg,Lighthouse,54.186291,15.554779,Kołobrzeg
7,NAKAMAL BAR & GRILL,Restaurant,54.184355,15.556195,Kołobrzeg
8,Portowa 28 Cafe,Café,54.183742,15.555946,Kołobrzeg
9,Plaża Centralna,Beach,54.186630,15.562654,Kołobrzeg


### Ustronie Morskie

In [232]:
i=5
# listing the venues as pandas dataframe
venues = json_data['TownJson'][i]['response']['groups'][0]['items']
UST_nearby_venues = json_normalize(venues) # flatten JSON
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng','venue.location.city']
UST_nearby_venues = UST_nearby_venues.loc[:, filtered_columns]
# filter the category for each row
UST_nearby_venues['venue.categories'] = UST_nearby_venues.apply(get_category_type, axis=1)
# clean columns
UST_nearby_venues.columns = [col.split(".")[-1] for col in UST_nearby_venues.columns]
#Substituting NaN 
UST_nearby_venues['city'] = UST_nearby_venues['city'].fillna('Ustronie Morskie')
UST_nearby_venues

,name,categories,lat,lng,city
0,Boulevard Ustronie Morskie,Hotel,54.216763,15.757094,Ustronie Morskie
1,Plaża,Beach,54.215262,15.750602,Ustronie Morskie
2,Biedronka,Market,54.215967,15.753675,Ustronie Morskie
3,Kabaczek,Restaurant,54.213929,15.750532,Ustronie Morskie
4,Hotel&Spa Lambert,Hotel Pool,54.217418,15.759632,Ustronie Morskie
5,Molo,Pier,54.214802,15.747939,Ustronie Morskie
6,Molo. Ustronie Morskie,Pier,54.216116,15.747491,Ustronie Morskie
7,Busola Cafe,Café,54.215117,15.744407,Ustronie Morskie


### Mielno

In [211]:
i=6
# listing the venues as pandas dataframe
venues = json_data['TownJson'][i]['response']['groups'][0]['items']
MIEL_nearby_venues = json_normalize(venues) # flatten JSON
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng','venue.location.city']
MIEL_nearby_venues = MIEL_nearby_venues.loc[:, filtered_columns]
# filter the category for each row
MIEL_nearby_venues['venue.categories'] = MIEL_nearby_venues.apply(get_category_type, axis=1)
# clean columns
MIEL_nearby_venues.columns = [col.split(".")[-1] for col in MIEL_nearby_venues.columns]
#Substituting NaN 
MIEL_nearby_venues['city'] = MIEL_nearby_venues['city'].fillna('Mielno')
MIEL_nearby_venues

,name,categories,lat,lng,city
0,Dune Restaurant Cafe & Lounge,Restaurant,54.263891,16.060677,Mielno
1,Mielno Plaża,Beach,54.263595,16.057034,Mielno
2,Mielno,City,54.260947,16.054370,Mielno
3,Meduza Hotel Restauracja,Restaurant,54.261928,16.050417,Mielno
4,Biedronka,Market,54.258800,16.054358,Mielno
5,Cafe Galeria,Café,54.262063,16.050771,Mielno


### Darłowo


In [266]:
i=7
# listing the venues as pandas dataframe
venues = json_data['TownJson'][i]['response']['groups'][0]['items']
DAR_nearby_venues = json_normalize(venues) # flatten JSON
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng','venue.location.city']
DAR_nearby_venues = DAR_nearby_venues.loc[:, filtered_columns]
# filter the category for each row
DAR_nearby_venues['venue.categories'] = DAR_nearby_venues.apply(get_category_type, axis=1)
# clean columns
DAR_nearby_venues.columns = [col.split(".")[-1] for col in DAR_nearby_venues.columns]
#Substituting NaN 
DAR_nearby_venues['city'] = DAR_nearby_venues['city'].fillna('Darłowo')
DAR_nearby_venues

,name,categories,lat,lng,city
0,Zamek Książąt Pomorskich w Darłowie,History Museum,54.420006,16.411331,Darłowo
1,Lidl,Supermarket,54.420154,16.402834,Darłowo
2,BP,Gas Station,54.414298,16.401759,Darłowo
3,Rynek,Plaza,54.422164,16.410978,Darłowo
4,Totolotek,Betting Shop,54.420831,16.416989,Darłowo
5,Darłowo,Train Station,54.417032,16.404476,Darłowo
6,Wild Bean Cafe,Hot Dog Joint,54.414428,16.402038,Darłowo
7,Port Darłówko,Harbor / Marina,54.422699,16.390377,Darłowo


In [268]:
print(DAR_nearby_venues)
#Joining all individual resort dfs into one
nearby_venues = pd.concat([SWI_nearby_venues,MIE_nearby_venues,DZI_nearby_venues, REW_nearby_venues, KOL_nearby_venues, UST_nearby_venues, MIEL_nearby_venues, DAR_nearby_venues], ignore_index=True)
nearby_venues

                                  name       categories        lat        lng  \
0  Zamek Książąt Pomorskich w Darłowie   History Museum  54.420006  16.411331   
1                                 Lidl      Supermarket  54.420154  16.402834   
2                                   BP      Gas Station  54.414298  16.401759   
3                                Rynek            Plaza  54.422164  16.410978   
4                            Totolotek     Betting Shop  54.420831  16.416989   
5                              Darłowo    Train Station  54.417032  16.404476   
6                       Wild Bean Cafe    Hot Dog Joint  54.414428  16.402038   
7                        Port Darłówko  Harbor / Marina  54.422699  16.390377   

      city  
0  Darłowo  
1  Darłowo  
2  Darłowo  
3  Darłowo  
4  Darłowo  
5  Darłowo  
6  Darłowo  
7  Darłowo  


,name,categories,lat,lng,city
0,Plac Wolności,Plaza,53.909011,14.248993,Świnoujście
1,Promenada,Plaza,53.918722,14.259256,Świnoujście
2,Hampton by Hilton,Hotel,53.913719,14.236786,Świnoujście
3,Plaża Miejska,Beach,53.921737,14.253880,Świnoujście
4,Magiczna Spiżarnia,Polish Restaurant,53.907843,14.255107,Świnoujście
5,Amsterdam Café,Restaurant,53.919272,14.256897,Świnoujście
6,Radisson Blu Resort Świnoujście,Resort,53.919284,14.260361,Świnoujście
7,Park Zdrojowy,Park,53.912908,14.258346,Świnoujście
8,Neptun,Eastern European Restaurant,53.909136,14.249110,Świnoujście
9,Mila,Restaurant,53.919168,14.258201,Świnoujście


# 3. Methodology and Exploratory Data Analysis

### 3.1. Summary of italian restaurants per resort

In [241]:
print(towns)

['Świnoujście', 'Międzyzrdoje', 'Dziwnów', 'Rewal', 'Kołobrzeg', 'Ustronie Morskie', 'Mielno', 'Darłowo', 'Postomino']


In [269]:
#Select Italian restaurants
print("Świnoujście: \n", SWI_nearby_venues[SWI_nearby_venues['categories']=='Italian Restaurant'])
print("Międzyzrdoje: \n", MIE_nearby_venues[MIE_nearby_venues['categories']=='Italian Restaurant'])
print("Dziwnów: \n", DZI_nearby_venues[DZI_nearby_venues['categories']=='Italian Restaurant'])
print("Rewal: \n",REW_nearby_venues[REW_nearby_venues['categories']=='Italian Restaurant'])
print("Kołobrzeg: \n",KOL_nearby_venues[KOL_nearby_venues['categories']=='Italian Restaurant'])
print("Ustronie Morskie: \n",UST_nearby_venues[UST_nearby_venues['categories']=='Italian Restaurant'])
print("Mielno: \n",MIEL_nearby_venues[MIEL_nearby_venues['categories']=='Italian Restaurant'])
print("Darłowo: \n",DAR_nearby_venues[DAR_nearby_venues['categories']=='Italian Restaurant'])

Świnoujście: 
 Empty DataFrame
Columns: [name, categories, lat, lng, city]
Index: []
Międzyzrdoje: 
                               name          categories        lat        lng  \
10  Dolce Vita Ristorante Italiana  Italian Restaurant  53.929005  14.450672   

            city  
10  Międzyzdroje  
Dziwnów: 
 Empty DataFrame
Columns: [name, categories, lat, lng, city]
Index: []
Rewal: 
 Empty DataFrame
Columns: [name, categories, lat, lng, city]
Index: []
Kołobrzeg: 
                    name          categories        lat        lng       city
18  Restauracja Portowa  Italian Restaurant  54.186564  15.555855  Kołobrzeg
Ustronie Morskie: 
 Empty DataFrame
Columns: [name, categories, lat, lng, city]
Index: []
Mielno: 
 Empty DataFrame
Columns: [name, categories, lat, lng, city]
Index: []
Darłowo: 
 Empty DataFrame
Columns: [name, categories, lat, lng, city]
Index: []


### 3.1. Map of each resort showing Restaurants disribution
#### 3.1.1. Map of Świnoujście

In [1]:
# create map of Świnoujście using latitude and longitude values
lat_Swinoujscie = 53.9097477
lon_Swinoujscie = 14.2510703

#create map
map_swino = folium.Map(location=[lat_Swinoujscie, lon_Swinoujscie], zoom_start=13)

# add markers to map
for lat, lng, name, categories in zip(nearby_venues['lat'], nearby_venues['lng'], nearby_venues['name'], nearby_venues['categories']):
    label = '{}, {}'.format(name, categories)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_swino)  



map_swino

NameError: name 'folium' is not defined

In [271]:
nearby_venues.shape

(121, 5)

### 3.2. Map of Międzyzdroje


In [ ]:
locdata[locdata['Address']=='Międzyzdroje, Poland']['Latitude']

In [ ]:
# create map of Świnoujście using latitude and longitude values
lat = locdata['Latitude']
lng = 14.2510703

#create map
map_swino = folium.Map(location=[lat_Swinoujscie, lon_Swinoujscie], zoom_start=13)

# add markers to map
for lat, lng, name, categories in zip(nearby_venues['lat'], nearby_venues['lng'], nearby_venues['name'], nearby_venues['categories']):
    label = '{}, {}'.format(name, categories)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_swino)  



map_swino

# 4. Results

# 5. Discussion

# 6. Conclusion / future developments

It could be beneficial to consider also data which will be collected from a newly opened places. Introducing a starting date of a venue to the dataset, could allow modelling of real success or failures of specific kinds of restaurants in certain locations.
As an enhancement to this project a similar code could be used to analyse situation in central-eastern seaside resorts.

